# Start

Import all libraries

In [1]:
!pip install sktime
!pip install neuralforecast
!pip install pmdarima
!pip install tbats

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.neuralforecast import NeuralForecastLSTM
from sktime.split import temporal_train_test_split
from sktime.performance_metrics.forecasting import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sktime.forecasting.model_selection import ForecastingGridSearchCV
from sktime.split import ExpandingWindowSplitter
from sktime.forecasting.arima import ARIMA
from sktime.forecasting.tbats import TBATS
import joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.9/36.9 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.5/141.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Load the data

In [2]:
dataset = pd.read_csv('/content/sample_data/weatherHistory.csv')
dataset

,Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-01-04 00:00:00+02:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-01-04 01:00:00+02:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-01-04 02:00:00+02:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-01-04 03:00:00+02:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-01-04 04:00:00+02:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.
...,...,...,...,...,...,...,...,...,...,...,...,...
37981,2016-09-09 19:00:00+02:00,Partly Cloudy,rain,26.016667,26.016667,0.43,10.9963,31.0,16.1000,0.0,1014.36,Partly cloudy starting in the morning.
37982,2016-09-09 20:00:00+02:00,Partly Cloudy,rain,24.583333,24.583333,0.48,10.0947,20.0,15.5526,0.0,1015.16,Partly cloudy starting in the morning.
37983,2016-09-09 21:00:00+02:00,Partly Cloudy,rain,22.038889,22.038889,0.56,8.9838,30.0,16.1000,0.0,1015.66,Partly cloudy starting in the morning.
37984,2016-09-09 22:00:00+02:00,Partly Cloudy,rain,21.522222,21.522222,0.60,10.5294,20.0,16.1000,0.0,1015.95,Partly cloudy starting in the morning.


Simple data pre-processing

In [3]:
# rename column
dataset = dataset.rename(columns={'Date': 'Datetime'})

# convert Date Format
dataset['Datetime'] = pd.to_datetime(dataset['Datetime'].astype(str).str.replace(r'\+02:00', '', regex=True), errors='coerce')

# drop Unnecessary Column ( "Loud Column has 0 values")
if 'Loud Cover' in dataset.columns:
  dataset.drop(columns=['Loud Cover'], inplace=True)

# drop daily summary column as it has overlapping data
if 'Daily Summary' in dataset.columns:
  dataset.drop(columns=['Daily Summary'], inplace=True)

# convert categorical valus into numerical values
leSummary = LabelEncoder()
leSummary.fit(dataset['Summary'])
replaceSummary = leSummary.transform(dataset['Summary'])
dataset['Summary'] = replaceSummary
lePrecipType = LabelEncoder()
lePrecipType.fit(dataset['Precip Type'])
replacePrecipType = lePrecipType.transform(dataset['Precip Type'])
dataset['Precip Type'] = replacePrecipType

# duplciate datatime values need to be dropped
dataset = dataset.drop_duplicates(subset=['Datetime'])
# set the datatime as the index value
dataset = dataset.set_index('Datetime').asfreq('h')

# normalise values using min max scaling
dataset = (dataset - dataset.mean()) / dataset.std()

dataset

,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Pressure (millibars)
Datetime,,,,,,,,,
2006-01-04 00:00:00,0.601503,-0.038029,-1.208642,-1.410715,1.049551,0.651996,0.530927,1.208981,0.091871
2006-01-04 01:00:00,0.601503,-0.038029,-1.225515,-1.432987,0.903921,0.674801,0.601637,1.208981,0.097043
2006-01-04 02:00:00,0.090133,-0.038029,-1.222301,-1.135777,1.049551,-0.951943,0.115506,0.944657,0.100250
2006-01-04 03:00:00,0.601503,-0.038029,-1.379776,-1.610391,0.758292,0.649462,0.690025,1.208981,0.105113
2006-01-04 04:00:00,0.090133,-0.038029,-1.312287,-1.467546,0.758292,0.168027,0.601637,1.208981,0.106147
...,...,...,...,...,...,...,...,...,...
2016-12-10 19:00:00,-0.165553,-0.038029,-1.247208,-1.530521,1.098094,1.376683,0.990542,-2.437702,0.054317
2016-12-10 20:00:00,-0.165553,-0.038029,-1.263277,-1.558936,1.098094,1.472970,0.946349,-1.155244,0.061662
2016-12-10 21:00:00,-0.165553,-0.038029,-1.300235,-1.613463,0.952465,1.586994,0.928671,-0.567859,0.068076


Set our forecasting horizon

In [4]:
forecast_horizon = ForecastingHorizon(np.arange(1, 25))

A dataframe will be used to keep track of all ML scores

In [5]:
scores = pd.DataFrame(columns=['Model', 'Mean Absolute Error', 'Root Mean Squared Error', 'Mean Absolute Percentage Error'])

Split the dataset up into a test train set

In [6]:
dataset = dataset.dropna()
dataset_train, dataset_test = temporal_train_test_split(dataset, fh=forecast_horizon)
dataset_train_features = dataset_train.loc[:, ['Summary',	'Precip Type',	'Humidity',	'Wind Speed (km/h)',	'Wind Bearing (degrees)',	'Visibility (km)',	'Pressure (millibars)']]
dataset_train_features.index = dataset_train_features.index.to_period(freq="h")
dataset_train_features = dataset_train_features.dropna()
dataset_train_target = dataset_train['Temperature (C)']
dataset_train_target.index = dataset_train_target.index.to_period(freq="h")
dataset_train_target = dataset_train_target.dropna()

# Models

## LSTM with sktime

Grid search can be used to optimise the hyperparameters but it takes an extremely long time to do so.
<br>
Code:
<br>

use grid search to find the optimal hyperparameters
lstmParameters = {'encoder_n_layers' : [2]}
cv = ExpandingWindowSplitter(fh=forecast_horizon)

modelLSTM = ForecastingGridSearchCV(forecaster=NeuralForecastLSTM(), param_grid=lstmParameters, cv=cv)
modelLSTM.fit(y=dataset_train_target, X=dataset_train_features, fh=forecast_horizon)

print best parameter after tuning
print(modelLSTM.best_params_)
gridPredictionsLSTM = modelLSTM.predict(fh=forecast_horizon)
print(gridPredictionsLSTM)


Alternative to grid search is to do it manually, look at dissertation code, TBATS section for this.

In [7]:
# build the model
modelLSTM = NeuralForecastLSTM()
modelLSTM.fit(y=dataset_train_target, X=dataset_train_features, fh=forecast_horizon)

# make predictions
lstmPredictions = modelLSTM.predict(fh=forecast_horizon)

# calculate scores based off predictions
lstmMAE = mean_absolute_error(dataset_test['Temperature (C)'], lstmPredictions)
lstmRMSE = mean_squared_error(dataset_test['Temperature (C)'], lstmPredictions, square_root=True)
lstmMAPE = mean_absolute_percentage_error(dataset_test['Temperature (C)'], lstmPredictions)

# save scores to panda dataframe
scores = scores._append({'Model': 'LSTM',
                         'Mean Absolute Error': lstmMAE,
                         'Root Mean Squared Error': lstmRMSE,
                         'Mean Absolute Percentage Error' : lstmMAPE},
                        ignore_index=True)
#Print Scores
print(scores)

# save model for use later
joblib.dump(modelLSTM, 'LSTM_Model')

/usr/local/lib/python3.11/dist-packages/neuralforecast/common/_base_model.py:138: UserWarning: Input size too small. Automatically setting input size to 3 * horizon = 72
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/neuralforecast/common/_base_model.py:146: UserWarning: Inference input size too small. Automatically setting inference input size to input_size = 72
  warnings.warn(
INFO:lightning_fabric.utilities.seed:Seed set to 1
/usr/local/lib/python3.11/dist-packages/neuralforecast/models/lstm.py:161: UserWarning: context_size is deprecated and will be removed in future versions.
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1000` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

<ipython-input-7-c50998431e30>:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  scores = scores._append({'Model': 'LSTM',


  Model  Mean Absolute Error  Root Mean Squared Error  \
0  LSTM              1.01438                  1.17958   

   Mean Absolute Percentage Error  
0                        0.786765  


['LSTM_Model']

## SARIMA with sktime

Auto ARIMA from sktime to automatically adjust parameters for the model but it takes way too long to use.

An "X" value is not passed in here as it caused errors. Model will be better if this can be fixed. Also model parameters need to be adjusted.

In [8]:
# build the model
modelSARIMA = ARIMA(order=(1, 1, 0),
                    seasonal_order=(0, 1, 0, 12))

# fit the model
modelSARIMA.fit(y=dataset_train_target, X=dataset_train_features, fh=forecast_horizon)

#checks on X features
print(dataset_train_features.shape)  # Should have rows
print(dataset_test.shape)            # Should have 24 rows
print(dataset_train_target.shape)    # Should have rows

#Possible fix
# Prepare test features (exogenous variables for the forecast horizon)
dataset_test_features = dataset_test.loc[:, ['Summary', 'Precip Type', 'Humidity', 'Wind Speed (km/h)',
                                            'Wind Bearing (degrees)', 'Visibility (km)', 'Pressure (millibars)']]
dataset_test_features.index = dataset_test_features.index.to_period(freq="h")

# End of possible FIX

# make predictions and save prediction
sarimaPredictions = modelSARIMA.predict(fh=forecast_horizon, X=dataset_test_features)

# calculate scores based off predictions
sarimaMAE = mean_absolute_error(dataset_test['Temperature (C)'], sarimaPredictions)
sarimaRMSE = mean_squared_error(dataset_test['Temperature (C)'], sarimaPredictions, square_root=True)
sarimaMAPE = mean_absolute_percentage_error(dataset_test['Temperature (C)'], sarimaPredictions)

# save scores to panda dataframe
scores = scores._append({'Model': 'SARIMA',
                         'Mean Absolute Error': sarimaMAE,
                         'Root Mean Squared Error': sarimaRMSE,
                         'Mean Absolute Percentage Error' : sarimaMAPE},
                        ignore_index=True)

# save model for use later
joblib.dump(modelSARIMA, 'SARIMA_Model')

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


(22134, 7)
(24, 9)
(22134,)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


['SARIMA_Model']

## TBATS with sktime

Using the best

In [ ]:
# build the model
modelTBATS = TBATS(use_box_cox=True,
                   box_cox_bounds=False,
                   use_trend=False,
                   use_damped_trend=True,
                   use_arma_errors=True,
                   show_warnings=False,
                   sp=24)
modelTBATS.fit(y=dataset_train_target, X=dataset_train_features, fh=forecast_horizon)

# make predictions
tbatsPredictions = modelTBATS.predict(fh=forecast_horizon)

# calculate scores based off predictions
tbatsMAE = mean_absolute_error(dataset_test['Temperature (C)'], tbatsPredictions)
tbatsRMSE = mean_squared_error(dataset_test['Temperature (C)'], tbatsPredictions, square_root=True)
tbatsMAPE = mean_absolute_percentage_error(dataset_test['Temperature (C)'], tbatsPredictions)

# save scores to panda dataframe
scores = scores._append({'Model': 'TBATS',
                         'Mean Absolute Error': tbatsMAE,
                         'Root Mean Squared Error': tbatsRMSE,
                         'Mean Absolute Percentage Error' : tbatsMAPE},
                        ignore_index=True)

# save model for use later
joblib.dump(modelTBATS, 'TBATS_Model')

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

# Results

Print out all the scores

In [ ]:
print(scores)

Plot Results onto a graph

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(scores['Model'], scores['Mean Absolute Error'])
plt.title('MAE of different ML models')
plt.xlabel('Models')
plt.ylabel('MAE scores')
plt.show()